# Data Science Capstone Final Project:
### Exploring the surrounding areas of schools in the capital of Sweden, Stockholm

#### Introduction/Business Problem

An area with schools can be appealing to many businesses that not only want to expand, but also increase their profits. After a long day at school, the students might decide to go for food with friends or family. A school with a lot of students for instance, could be a target for building a new bakery or a restaurant. 

At the same time, getting information about what exists around a school can be signifant for parents, who want to choose a school for their kids that provides a lot of places where their kids and themselves can relax before or after school.

My goal with this project is to find which are the most common venues next to schools around Stockholm and if there is any correlation with the number of enrolled students. Furthermore, I would like to investigate if there is a specific pattern with the most common venues that are somehow are related to presence of schools. 

#### Data
The school data were provided from an open source data portal: https://dataportalen.stockholm.se/. They were in a form of a dbf database, so the simpledbf library was utilized so that to create a pandas DataFrame for further analysis. 

In [2]:
import pandas as pd
from simpledbf import Dbf5


In [3]:
dbf = Dbf5('alla_skolor_1920_elevantal.dbf')

df = dbf.to_dataframe()

In [4]:
df.head()

,SDO,REGI,SKOLENHETS,Skolenhe_1,SKOLA,Arskurs,Elevanal19,Särskola,Särskola_,X_North,Y_East,NOTE,ADRESS,POSTNR,POSTORT,BESÖKSADR,Kommentar
0,Bromma,Kommunala grundskolor,44673074,Abrahamsbergsskolan,Abrahamsbergsskolan,F-9,678,0,0,6579984.0,147176.0,NaN,Gustav III:s väg 2-10,16830,BROMMA,NaN,NaN
1,Norrmalm,Kommunala grundskolor,99648792,Adolf Fredriks musikklasser,Adolf Fredriks Musikklasser,4-9,1120,0,0,6580259.0,152895.0,NaN,Box 6267,10231,STOCKHOLM,NaN,NaN
2,Bromma,Kommunala grundskolor,41285467,Adolfsbergsskolan,Adolfsbergsskolan,F-1,132,0,0,6582511.0,147360.0,NaN,Gårdsfogdevägen 18 B,16867,BROMMA,NaN,NaN
3,Rinkeby-Kista,Fristående grundskolor,17526606,Akademiska Skolan i Kista,Akademiska skolan i Kista-Husby,F-6,35,0,0,6588485.0,145913.0,NaN,Lofotengatan 32-34,16433,KISTA,NaN,NaN
4,Rinkeby-Kista,Kommunala grundskolor,37796564,Akalla grundskola F-9,Akalla grundskola F-9,F-9,498,1,0,6588999.0,144905.0,NaN,Mariehamnsgatan 3-5,16471,KISTA,NaN,NaN


In [5]:
df.shape

(375, 17)

In [6]:
len(df.SKOLA.unique())

372

The dataFrame has 375 entries (372 different schools) and 17 features (some features will be filtered out later). 
1. The most important ones are SDO- which is the region where the school is located, 
2. REGI- if the school is municipal or stand-alone,
3. SKOLA-The name of the school, 
4. Årskurs- the school grade that the data come from, 
5. Elevanal19- the number of enrolled students for 2019-20, 
6. X_North & Y_East- the coordinates in SWEREF99 1800 coordinate system (Transformation to WGS84 standard coordination system is necessary to work both with the Foursquare API and Folium library)